In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
inflight_sat = pd.read_csv('Survey data_Inflight Satisfaction Score.csv')

In [3]:
inflight_sat.head(2)

,flight_number,origin_station_code,destination_station_code,record_locator,scheduled_departure_date,question_text,score,satisfaction_type,driver_sub_group1,driver_sub_group2,...,equipment_type_code,ua_uax,actual_flown_miles,haul_type,departure_gate,arrival_gate,international_domestic_indicator,response_group,media_provider,hub_spoke
0,3802,MKX,ORX,CYXXJJ,9/1/2022,How satisfied were you with the food & beverag...,2,Dissatisfied,food & beverage,food and beverage satisfaction,...,CRZ,UAX,67,Short,C12,F10,Domestic,non-member,NaN,spoke departure
1,4645,COX,DEX,DRXXMH,9/1/2022,How satisfied were you with the food & beverag...,5,Satisfied,food & beverage,food and beverage satisfaction,...,E75,UAX,72,Short,5,B60,Domestic,member,GOGO,spoke departure


In [47]:
inflight_sat['international_domestic_indicator'].unique()

array(['Domestic', 'International'], dtype=object)

In [4]:
inflight_sat.columns

Index(['flight_number', 'origin_station_code', 'destination_station_code',
       'record_locator', 'scheduled_departure_date', 'question_text', 'score',
       'satisfaction_type', 'driver_sub_group1', 'driver_sub_group2',
       'arrival_delay_minutes', 'arrival_delay_group', 'cabin_code_desc',
       'cabin_name', 'entity', 'number_of_legs', 'seat_factor_band',
       'loyalty_program_level', 'generation', 'fleet_type_description',
       'fleet_usage', 'equipment_type_code', 'ua_uax', 'actual_flown_miles',
       'haul_type', 'departure_gate', 'arrival_gate',
       'international_domestic_indicator', 'response_group', 'media_provider',
       'hub_spoke'],
      dtype='object')

In [57]:
inflight_sat['international_domestic_indicator'].unique()

array(['Domestic', 'International'], dtype=object)

In [56]:
df = inflight_sat[['flight_number','origin_station_code','destination_station_code','score','satisfaction_type','arrival_delay_minutes','actual_flown_miles','arrival_delay_group','departure_gate','international_domestic_indicator']]

In [63]:
df['satisfaction_type'].replace('Dissatisfied', 0, inplace=True)
df['satisfaction_type'].replace('Satisfied', 1, inplace=True)
df['arrival_delay_group'].replace('Delayed', 0, inplace=True)
df['arrival_delay_group'].replace('Early & Ontime', 1, inplace=True)
df['international_domestic_indicator'].replace({'Domestic':0,'International':1}, inplace=True)
df.dropna(inplace=True)

C:\Users\rewan\AppData\Local\Temp\ipykernel_22328\4128219850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['satisfaction_type'].replace('Dissatisfied', 0, inplace=True)
C:\Users\rewan\AppData\Local\Temp\ipykernel_22328\4128219850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['satisfaction_type'].replace('Satisfied', 1, inplace=True)
C:\Users\rewan\AppData\Local\Temp\ipykernel_22328\4128219850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [64]:
df

,flight_number,origin_station_code,destination_station_code,score,satisfaction_type,arrival_delay_minutes,actual_flown_miles,arrival_delay_group,departure_gate,international_domestic_indicator
0,3802,MKX,ORX,2,0.0,-24,67,1,C12,0
1,4645,COX,DEX,5,1.0,10,72,0,5,0
2,5236,COX,DEX,5,1.0,-8,72,1,7,0
3,4686,DEX,COX,3,0.0,-11,72,1,B82,0
4,671,DEX,COX,5,1.0,18,72,0,B24,0
...,...,...,...,...,...,...,...,...,...,...
47066,189,EWX,JNX,4,1.0,-3,7989,1,C126,1
47067,188,JNX,EWX,3,0.0,-22,7989,1,A11,1
47069,2,SFX,SIX,4,1.0,-16,8440,1,G9,1
47070,2,SFX,SIX,4,1.0,-16,8440,1,G9,1


In [65]:
def calc_ifs(df,column):
    gb = df.groupby(column)
    sat = gb['satisfaction_type'].sum()
    total = gb['satisfaction_type'].count()
    miles = gb['actual_flown_miles'].mean()
    delay = gb['arrival_delay_minutes'].mean()
    inter = gb['international_domestic_indicator'].sum()
    dom = gb['international_domestic_indicator'].count() - inter
    ifs = sat/total*100
    return ifs, miles, delay,total, dom, inter

In [70]:
ifs, miles, delay, total, dom, inter = calc_ifs(df,'flight_number')
result = pd.DataFrame({'ifs':ifs, 'delay':delay,'domestic':dom, 'international':inter})
result[total>50].to_csv('flight_no_ifs.csv')

In [55]:
ifs, miles, delay, total = calc_ifs(df,'departure_gate')
result = pd.DataFrame({'ifs':ifs, 'delay':delay})
result[total>50].to_csv('departuregate_ifs.csv')

In [43]:
domestic = df[inflight_sat['international_domestic_indicator']=='Domestic']

C:\Users\rewan\AppData\Local\Temp\ipykernel_22328\1936940588.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  domestic = df[inflight_sat['international_domestic_indicator']=='Domestic']


In [46]:
ifs, miles, delay, total = calc_ifs(domestic,'flight_number')
result = pd.DataFrame({'ifs':ifs, 'delay':delay})
result[total>50].to_csv('domestic_flight_no_ifs.csv')

In [48]:
international = df[inflight_sat['international_domestic_indicator']=='International']

C:\Users\rewan\AppData\Local\Temp\ipykernel_22328\337549563.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  international = df[inflight_sat['international_domestic_indicator']=='International']


In [49]:
ifs, miles, delay, total = calc_ifs(international,'flight_number')
result = pd.DataFrame({'ifs':ifs, 'delay':delay})
result[total>50].to_csv('international_flight_no_ifs.csv')

In [10]:
gb = df.groupby(['flight_number'])
sat = gb['satisfaction_type'].sum()
total = gb['satisfaction_type'].count()
miles = gb['actual_flown_miles'].mean()
delay = gb['arrival_delay_minutes'].mean()
delayp = gb['arrival_delay_group'].sum()/gb['arrival_delay_group'].count()*100
ifs = sat/total*100

In [19]:
ifs[total>50].to_csv('flight_no_ifs.csv')

In [17]:
ifs.to_csv('flight_no_ifs.csv')

In [8]:
gb = df.groupby(['origin_station_code','destination_station_code'])
sat = gb['satisfaction_type'].sum()
total = gb['satisfaction_type'].count()
miles = gb['actual_flown_miles'].mean()
delay = gb['arrival_delay_minutes'].mean()
delayp = gb['arrival_delay_group'].sum()/gb['arrival_delay_group'].count()*100
ifs = sat/total*100